#MountDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# Working Directory
os.chdir('/content/drive/MyDrive/ML_Colab/AV')

#UnZip

In [ ]:
# Function for unzip the dataset in the selected folder
import zipfile
from tqdm import tqdm
import os

def unzip_file(zip_path, extract_to):
    """
    Unzip a file to the specified directory using tqdm for progress tracking.

    Parameters:
    - zip_path (str): Path to the zip file.
    - extract_to (str): Directory where the contents will be extracted.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get the total number of entries in the zip file for tqdm
        total_entries = sum(1 for _ in zip_ref.infolist())

        # Set up tqdm for progress tracking
        with tqdm(total=total_entries, desc="Extracting", unit="file") as pbar:
            for entry in zip_ref.infolist():
                zip_ref.extract(entry, extract_to)
                pbar.update(1)

# Usage example
zip_file_path = '/content/drive/MyDrive/ML_Colab/AV/PAR_DATASET/Validation/validation_set.zip'
extract_to_directory = '/content/drive/MyDrive/ML_Colab/AV/Dataset'

unzip_file(zip_file_path, extract_to_directory)


#ParDataset creation

In [ ]:
# Custom Dataset class for creating a dataset of pairs (Image,labels)
# During the creation are selected only the images with all labels, and are applied
# data transformation for the compatibility with the chosed pretrained network
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import torch

class CustomDataset(Dataset):
    def __init__(self, root_dir, txt_file, transform=None, max_images=1000):
        self.root_dir = root_dir
        self.transform = transform
        self.max_images = max_images

        with open(txt_file, 'r') as f:
            lines = f.readlines()

        self.data = []
        loaded_images = 0

        for line in lines:
            line = line.strip().split(',')
            image_name = line[0]

            # Prendi solo le etichette che non sono -1
            if int(line[1]) != -1 and int(line[2]) != -1 and int(line[3]) != -1 and int(line[4]) != -1 and int(line[5]) != -1:
                labels = [int(x) for x in line[1:6]]  # Converti le etichette in int e crea una lista
                # Sottrai -1 solo alle prime due etichette
                labels[0] -= 1
                labels[1] -= 1

                labels = torch.tensor(labels)  # Converti la lista di etichette in un tensore
            else:
                continue  # Passa alla successiva riga se ci sono etichette -1 o raggiunto il limite massimo di immagini

            img_path = os.path.join(self.root_dir, image_name)

            try:
                image = Image.open(img_path)

                # Applica la trasformazione qui
                if self.transform:
                    image = self.transform(image)

            except Exception as e:
                # print(f"Errore durante l'apertura o trasformazione dell'immagine {img_path}: {e}")
                continue

            # Usa tutte le etichette valide
            self.data.append((image, labels))
            loaded_images += 1

            if loaded_images == self.max_images:
                break

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, labels = self.data[idx]
        return image, labels




In [ ]:
# Training Dataset of 5000 samples
# Directory contenente le immagini
root_dir = '/content/drive/MyDrive/ML_Colab/AV/Dataset/training_set'
label_file = '/content/drive/MyDrive/ML_Colab/AV/PAR_DATASET/Training/training_set.txt'

# Esempio di trasformazioni che puoi applicare alle immagini (modifica a seconda delle tue esigenze)
import torchvision.transforms as transforms

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Cambia la dimensione dell'immagine a 224x224
    transforms.ToTensor(),  # Converte l'immagine in un tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizza l'immagine
])

# Creazione dell'istanza del dataset con massimo 1000 immagini
custom_dataset = CustomDataset(root_dir, label_file, transform=data_transform, max_images=5000)

In [ ]:
# Script to calculate the number of elements for each class used to calculate the weighted losses


# Initialize color counts
black, blue, brown, gray, green, orange, pink, purple, red, white, yellow = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Initialize label counts for other attributes
l_black, l_blue, l_brown, l_gray, l_green, l_orange, l_pink, l_purple, l_red, l_white, l_yellow = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
male, female = 0, 0
bag, no_bag = 0, 0
hat, no_hat = 0, 0

# Iterate through the dataset
for i in range(len(custom_dataset)):
    image, labels = custom_dataset[i]

    # Update color counts
    if labels[0] == 0:
        black += 1
    elif labels[0] == 1:
        blue += 1
    elif labels[0] == 2:
        brown += 1
    elif labels[0] == 3:
        gray += 1
    elif labels[0] == 4:
        green += 1
    elif labels[0] == 5:
        orange += 1
    elif labels[0] == 6:
        pink += 1
    elif labels[0] == 7:
        purple += 1
    elif labels[0] == 8:
        red += 1
    elif labels[0] == 9:
        white += 1
    elif labels[0] == 10:
        yellow += 1

    # Update counts for other attributes
    if labels[1] == 0:
        l_black += 1
    elif labels[1] == 1:
        l_blue += 1
    elif labels[1] == 2:
        l_brown += 1
    elif labels[1] == 3:
        l_gray += 1
    elif labels[1] == 4:
        l_green += 1
    elif labels[1] == 5:
        l_orange += 1
    elif labels[1] == 6:
        l_pink += 1
    elif labels[1] == 7:
        l_purple += 1
    elif labels[1] == 8:
        l_red += 1
    elif labels[1] == 9:
        l_white += 1
    elif labels[1] == 10:
        l_yellow += 1

    if labels[2] == 0:
      male += 1
    if labels[2] == 1:
      female += 1

    if labels[3] == 0:
      no_bag += 1
    if labels[3] == 1:
      bag += 1

    if labels[4] == 0:
      no_hat += 1
    if labels[4] == 1:
      hat += 1

# Print the counts
print("Color Counts:")
print(f"Black: {black}, Blue: {blue}, Brown: {brown}, Gray: {gray}, Green: {green}, Orange: {orange}, Pink: {pink}, Purple: {purple}, Red: {red}, White: {white}, Yellow: {yellow}")

print("\nLabel Counts:")
print(f"L_Black: {l_black}, L_Blue: {l_blue}, L_Brown: {l_brown}, L_Gray: {l_gray}, L_Green: {l_green}, L_Orange: {l_orange}, L_Pink: {l_pink}, L_Purple: {l_purple}, L_Red: {l_red}, L_White: {l_white}, L_Yellow: {l_yellow}")
print(f"Male: {male}, Female: {female}")
print(f"Bag: {bag}, No Bag: {no_bag}")
print(f"Hat: {hat}, No Hat: {no_hat}")


Color Counts:
Black: 2814, Blue: 348, Brown: 111, Gray: 586, Green: 142, Orange: 43, Pink: 91, Purple: 71, Red: 334, White: 349, Yellow: 111

Label Counts:
L_Black: 3331, L_Blue: 1228, L_Brown: 23, L_Gray: 298, L_Green: 29, L_Orange: 1, L_Pink: 1, L_Purple: 0, L_Red: 16, L_White: 22, L_Yellow: 51
Male: 3782, Female: 1218
Bag: 942, No Bag: 4058
Hat: 130, No Hat: 4870


#Model Tuning

In [ ]:
# Definition of the multitask network built using the mobileNetV2 network as a shared basic architecture
# freezing the weights up to the second to last level and adding an output head for each task.
import torch
import torch.nn as nn
import torchvision.models as models

class MultiTaskMobileNetV2(nn.Module):
    def __init__(self, num_output_gender, num_output_hat, num_output_bag, num_output_top_color, num_output_bottom_color):
        super(MultiTaskMobileNetV2, self).__init__()

        # Load the pre-trained MobileNetV2 shared base
        self.base_model = models.mobilenet_v2(pretrained=True)

        # Freeze all layers up to the second-to-last layer
        for name, param in self.base_model.named_parameters():
            if not name.startswith('classifier') and not name.startswith('features.18'):  # Ignore the last fully connected layer
                param.requires_grad = False

        # Modify the last fully connected layer to match the tasks
        self.base_model.classifier[-1] = nn.Identity()

        # Add task-specific branches for each task
        self.gender_fc = nn.Linear(1280, num_output_gender)
        self.hat_fc = nn.Linear(1280, num_output_hat)
        self.bag_fc = nn.Linear(1280, num_output_bag)
        self.top_color_fc = nn.Linear(1280, num_output_top_color)
        self.bottom_color_fc = nn.Linear(1280, num_output_bottom_color)

    def forward(self, x):
        # Pass through the shared MobileNetV2 base
        shared_features = self.base_model(x)

        # Output for each task
        output_gender = self.gender_fc(shared_features)
        output_hat = self.hat_fc(shared_features)
        output_bag = self.bag_fc(shared_features)
        output_top_color = self.top_color_fc(shared_features)
        output_bottom_color = self.bottom_color_fc(shared_features)

        return output_gender, output_hat, output_bag, output_top_color, output_bottom_color




In [ ]:
# Model Creation
model = MultiTaskMobileNetV2(num_output_gender=1, num_output_hat=1, num_output_bag=1, num_output_top_color=11, num_output_bottom_color=11)
model

#Cross-validation and dataloaders

In [ ]:
from torch.utils.data import Subset, DataLoader
from torch.utils.data import random_split

# Function used for the creation of folder for K-cross validation
def cross_val_dataloaders(train_dataset, val_dataset=None, K=10, batch_size=32):
  if val_dataset is None:
    val_dataset = train_dataset

  dataloader_params = {"batch_size": batch_size, "num_workers": 2, "pin_memory": True}
  if K == 1:
          # If K = 1, perform a division of the data into training and validation using a specified proportion
          proportion = 0.2
          total_length = len(train_dataset)
          val_length = int(total_length * proportion)
          train_length = total_length - val_length

          train_dataset, val_dataset = random_split(train_dataset, [train_length, val_length])

          # Creazione dei DataLoader per training e validation
          train_fold = DataLoader(train_dataset, shuffle=True, **dataloader_params)
          val_fold = DataLoader(val_dataset, shuffle=False, **dataloader_params)

          return [train_fold], [val_fold], None
  else:
    indexes = torch.randperm(len(train_dataset)) % K # Random sequence of unique indices with respect to module K
    train_folds, val_folds = [], []
    for k in range(K):

        val_fold   = Subset(val_dataset,   (indexes==k).nonzero().squeeze())
        train_fold = Subset(train_dataset, (indexes!=k).nonzero().squeeze())

        val_fold   = DataLoader(val_fold,   shuffle=False, **dataloader_params) # Shuffle False because it is important in validation to maintain order to evaluate the model
        train_fold = DataLoader(train_fold, shuffle=True,  **dataloader_params) # Shuffle to true can help prevent the model from fitting to specific sequences and improve generalization

        val_folds.append(val_fold)
        train_folds.append(train_fold)

    return train_folds, val_folds, indexes

# Training monitoring

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook

def start_tensorboard(log_dir):
  writer = SummaryWriter(os.path.join("runs", log_dir))

  # run tensorboard in background
  ! killall tensorboard
  %load_ext tensorboard
  %tensorboard --logdir ./runs

  notebook.list() # View open TensorBoard instances

  return writer

#Training and validation

In [ ]:
import torch
# Definition of one epoch of trainig computing training e validation loss and accuracy
def one_epoch(model, lossFunctions, output_activations, optimizer, train_loader, val_loader, writer, epoch, device):
    model.to(device)
    model.train()

    i_start = epoch * len(train_loader)

    for i, (X, y) in tqdm(enumerate(train_loader), desc=f"Epoch {epoch} - Train"):
        if i == 0:
            writer.add_image('first_batch', torchvision.utils.make_grid(X.squeeze()))

        X = X.squeeze().to(device)
        y_top_color = y[:, 0].clone().to(device).long()
        y_bottom_color = y[:, 1].clone().detach().to(device).long()
        y_gender = y[:, 2].clone().detach().to(device).float()
        y_hat = y[:, 3].clone().detach().to(device).float()
        y_bag = y[:, 4].clone().detach().to(device).float()



        optimizer.zero_grad()

        # Forward pass
        o_gender, o_hat, o_bag, o_top_color, o_bottom_color = model(X)
        o_gender, o_hat, o_bag, o_top_color, o_bottom_color = output_activations[0](o_gender).squeeze(), output_activations[1](o_hat).squeeze(), output_activations[2](o_bag).squeeze(), output_activations[3](o_top_color).squeeze(), output_activations[4](o_bottom_color).squeeze()

        # Compute losses for each task
        l_gender = lossFunctions[0](o_gender, y_gender)
        l_hat = lossFunctions[1](o_hat, y_hat)
        l_bag = lossFunctions[2](o_bag, y_bag)
        l_top_color = lossFunctions[3](o_top_color, y_top_color)
        l_bottom_color = lossFunctions[4](o_bottom_color, y_bottom_color)

        # Compute the total loss (you may weigh the losses if needed)
        total_loss = l_gender + l_hat + l_bag + l_top_color + l_bottom_color

        # Backward pass
        total_loss.backward()
        optimizer.step()

        acc_g = ((o_gender.detach() > .5) == y_gender.detach()).float().mean()
        acc_h = ((o_hat.detach() > .5) == y_hat.detach()).float().mean()
        acc_b = ((o_bag.detach() > .5) == y_bag.detach()).float().mean()

        # Trova la classe predetta
        predicted_classes_tc = torch.argmax(o_top_color, dim=1)
        predicted_classes_bc = torch.argmax(o_bottom_color, dim=1)

        acc_tc = (predicted_classes_tc == y_top_color).float().mean()
        acc_bc = (predicted_classes_bc == y_bottom_color).float().mean()

        total_accuracy = (acc_g + acc_h + acc_b + acc_tc + acc_bc) / 5

        print("- batch loss and accuracy : {:.7f}\t{:.4f}".format(total_loss.detach().item(), total_accuracy))
        writer.add_scalar('train/loss', total_loss.detach().item(), i_start + i)
        writer.add_scalar('train/acc', total_accuracy, i_start + i)

    model.eval()  # Set up the model for evaluation
    with torch.no_grad():
        val_loss = []
        val_acc_g = []
        val_acc_h = []
        val_acc_b = []
        val_acc_tc = []
        val_acc_bc = []

        for X, y in tqdm(val_loader, desc="epoch {} - validation".format(epoch)):
            X = X.squeeze().to(device)
            y_top_color = y[:, 0].clone().to(device).long()
            y_bottom_color = y[:, 1].clone().detach().to(device).long()
            y_gender = y[:, 2].clone().detach().to(device).float()
            y_hat = y[:, 3].clone().detach().to(device).float()
            y_bag = y[:, 4].clone().detach().to(device).float()

            # Forward pass
            o_gender, o_hat, o_bag, o_top_color, o_bottom_color = model(X)
            o_gender, o_hat, o_bag, o_top_color, o_bottom_color = output_activations[0](o_gender).squeeze(), output_activations[1](o_hat).squeeze(), output_activations[2](o_bag).squeeze(), output_activations[3](o_top_color).squeeze(), output_activations[4](o_bottom_color).squeeze()

            # Compute losses for each task
            l_gender = lossFunctions[0](o_gender, y_gender)
            l_hat = lossFunctions[1](o_hat, y_hat)
            l_bag = lossFunctions[2](o_bag, y_bag)
            l_top_color = lossFunctions[3](o_top_color, y_top_color)
            l_bottom_color = lossFunctions[4](o_bottom_color, y_bottom_color)

            # Compute the total loss (you may weigh the losses if needed)
            total_loss = l_gender + l_hat + l_bag + l_top_color + l_bottom_color

            val_loss.append(total_loss)

            # Compute accuracies for each task
            acc_g = ((o_gender > 0.5) == y_gender).float().mean().item()
            acc_h = ((o_hat > 0.5) == y_hat).float().mean().item()
            acc_b = ((o_bag > 0.5) == y_bag).float().mean().item()
            acc_tc = (torch.argmax(o_top_color, dim=1) == y_top_color).float().mean().item()
            acc_bc = (torch.argmax(o_bottom_color, dim=1) == y_bottom_color).float().mean().item()

            val_acc_g.append(acc_g)
            val_acc_h.append(acc_h)
            val_acc_b.append(acc_b)
            val_acc_tc.append(acc_tc)
            val_acc_bc.append(acc_bc)

        val_loss = torch.stack(val_loss).mean().item()
        val_acc_g = sum(val_acc_g) / len(val_acc_g)
        val_acc_h = sum(val_acc_h) / len(val_acc_h)
        val_acc_b = sum(val_acc_b) / len(val_acc_b)
        val_acc_tc = sum(val_acc_tc) / len(val_acc_tc)
        val_acc_bc = sum(val_acc_bc) / len(val_acc_bc)
        total_accuracy = (val_acc_g + val_acc_h + val_acc_b + val_acc_tc + val_acc_bc) / 5

        print("Validation loss and accuracy : {:.7f}\t{:.4f}".format(val_loss, total_accuracy))
        writer.add_scalar('val/loss', val_loss, i_start + i)
        writer.add_scalar('val/acc', total_accuracy, i_start + i)

    return val_loss, total_accuracy


# Run the code

In [ ]:
import torch.nn as nn
import torch

# Weighted losses used to address class imbalance in the training data.
# By assigning different weights to different classes based on their distribution,
# the model is encouraged to focus more on under-represented classes during training.

# Definition of class distributions
class_counts_color = torch.tensor([black, blue, brown, gray, green, orange, pink, purple, red, white, yellow], dtype=torch.float)
class_counts_label = torch.tensor([l_black, l_blue, l_brown, l_gray, l_green, l_orange, l_pink, l_purple, l_red, l_white, l_yellow], dtype=torch.float)
class_counts_gender = torch.tensor([male, female], dtype=torch.float)
class_counts_bag = torch.tensor([bag, no_bag], dtype=torch.float)
class_counts_hat = torch.tensor([hat, no_hat], dtype=torch.float)

# Normalization of class distributions
class_weights_color = class_counts_color / class_counts_color.sum()
class_weights_label = class_counts_label / class_counts_label.sum()
class_weights_gender = class_counts_gender / class_counts_gender.sum()
class_weights_bag = class_counts_bag / class_counts_bag.sum()
class_weights_hat = class_counts_hat / class_counts_hat.sum()

# Definition of weighted losses
weighted_loss_color = nn.CrossEntropyLoss(weight=1 / class_weights_color)
weighted_loss_label = nn.CrossEntropyLoss(weight=1 / class_weights_label)
weighted_loss_gender = nn.BCEWithLogitsLoss(pos_weight=class_weights_gender[1] / class_weights_gender[0])
weighted_loss_bag = nn.BCEWithLogitsLoss(pos_weight=class_weights_bag[1] / class_weights_bag[0])
weighted_loss_hat = nn.BCEWithLogitsLoss(pos_weight=class_weights_hat[1] / class_weights_hat[0])





Parameter definition


In [ ]:
import torch
from torch.nn import BCELoss,Sigmoid

import torch.nn as nn

# Loss functions
binary_classification_loss = nn.BCELoss()
multiclass_classification_loss = nn.CrossEntropyLoss()

# Output layer for each task
output_activation_gender = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_hat = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_bag = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_top_color = nn.Softmax(dim=1)  # Funzione di attivazione Softmax per la classificazione multiclasse
output_activation_bottom_color = nn.Softmax(dim=1)  # Funzione di attivazione Softmax per la classificazione multiclasse

# List of loss
#lossFunctions = [binary_classification_loss, binary_classification_loss, binary_classification_loss, multiclass_classification_loss, multiclass_classification_loss]
# Loss weighted
lossFunctions = [weighted_loss_gender,weighted_loss_bag, weighted_loss_hat,weighted_loss_color,weighted_loss_label]
output_activations = [output_activation_gender, output_activation_hat, output_activation_bag, output_activation_top_color, output_activation_bottom_color]

batch_size = 64
lr = .001
momentum = .9
lambda_reg = 0

epochs = 40
early_stopping_patience = 4

K_cross_val = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# network parameters
num_output = 1 #Number of classes of the classification problem

# create output directory and logger
experiment_name = 'MTNN'
import os
os.makedirs(experiment_name)
writer = start_tensorboard(experiment_name)

Training

In [ ]:
import torch
import torchvision
from tqdm import tqdm


# Creating folders and saving the index in memory to optionally restore it
train_folds, val_folds, indexes = cross_val_dataloaders(custom_dataset,None, K_cross_val, batch_size)
torch.save(indexes, os.path.join(experiment_name, "cross-val-indexes.pt"))

# Initialize variables with relative size
val_losses = torch.zeros(epochs, K_cross_val)
val_accuracies = torch.zeros(epochs, K_cross_val)


for k in range(K_cross_val):

  # dataloader, network, optimizer for each fold
  train_loader, val_loader = train_folds[k], val_folds[k]
  model = MultiTaskMobileNetV2(num_output_gender=1, num_output_hat=1, num_output_bag=1, num_output_top_color=11, num_output_bottom_color=11)
  model = model.to(device)

  # Ottieni tutti i parametri della rete, inclusi quelli dei rami specifici per ciascun compito
  all_parameters = list(model.base_model.parameters()) + \
                  list(model.gender_fc.parameters()) + \
                  list(model.hat_fc.parameters()) + \
                  list(model.bag_fc.parameters()) + \
                  list(model.top_color_fc.parameters()) + \
                  list(model.bottom_color_fc.parameters())

  # Definisci l'ottimizzatore utilizzando tutti i parametri
  optimizer = torch.optim.SGD(all_parameters,
                              lr=lr,
                              weight_decay=lambda_reg,
                              momentum=momentum)



  # early stopping and best model saving
  early_stopping_counter = early_stopping_patience
  min_val_loss = 1e10

  # training and validation
  for e in range(epochs):
    print("FOLD {} - EPOCH {}".format(k, e))

    val_loss, val_accuracy = one_epoch(model, lossFunctions,output_activations, optimizer, train_loader, val_loader, writer,e,device)

    # store the validation metrics
    val_losses[e, k] = val_loss
    val_accuracies[e, k] = val_accuracy
    torch.save(val_losses, os.path.join(experiment_name,'val_losses.pth'))
    torch.save(val_accuracies, os.path.join(experiment_name,'val_accuracies.pth'))

    # save the best model and check the early stopping criteria
    if val_loss < min_val_loss: # save the best model
      min_val_loss = val_loss
      early_stopping_counter = early_stopping_patience # reset early stopping counter
      torch.save(model.state_dict(), os.path.join(experiment_name,'fold_{}_best_model.pth'.format(k)))
      print("- saved best model")

    if e>0: # early stopping counter update
      if val_losses[e, k] > val_losses[e-1, k]:
          early_stopping_counter -= 1 # update early stopping counter
      else:
          early_stopping_counter = early_stopping_patience # reset early stopping counter
    if early_stopping_counter == 0: # early stopping
        break

#Testing

In [ ]:
# Creation of Test Dataset of 500 samples
import torchvision.transforms as transforms

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Cambia la dimensione dell'immagine a 224x224
    transforms.ToTensor(),  # Converte l'immagine in un tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizza l'immagine
])

root_dir = '/content/drive/MyDrive/ML_Colab/AV/Dataset/validation_set'
label_file = '/content/drive/MyDrive/ML_Colab/AV/PAR_DATASET/Validation/validation_set.txt'


# Creazione dell'istanza del dataset con massimo 1000 immagini
Test_Set = CustomDataset(root_dir, label_file, transform=data_transform, max_images=500)

In [ ]:
# Calculation of the accuracy for each task
import torch
from torchvision import transforms
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Definisci il tuo modello MultiTaskMobileNetV2
model = MultiTaskMobileNetV2(num_output_gender=1, num_output_hat=1, num_output_bag=1, num_output_top_color=11, num_output_bottom_color=11)
weights_path = '/content/drive/MyDrive/ML_Colab/AV/MultiTask_w_loss/fold_0_best_model.pth'
model.load_state_dict(torch.load(weights_path))
model.eval()

# Livelli di output per ciascun compito
output_activation_gender = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_hat = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_bag = nn.Sigmoid()  # Funzione di attivazione sigmoide per la classificazione binaria
output_activation_top_color = nn.Softmax(dim=1)  # Funzione di attivazione Softmax per la classificazione multiclasse
output_activation_bottom_color = nn.Softmax(dim=1)  # Funzione di attivazione Softmax per la classificazione multiclasse
output_activations = [output_activation_gender, output_activation_hat, output_activation_bag, output_activation_top_color, output_activation_bottom_color]


correct_gender = 0
correct_hat = 0
correct_bag = 0
correct_top_color = 0
correct_bottom_color = 0

total_samples = len(Test_Set)

with torch.no_grad():

    for X, y in Test_Set:

        X = X.unsqueeze(0)
        y_top_color = y[0]
        y_bottom_color = y[1]
        y_gender = y[2]
        y_hat = y[3]
        y_bag = y[4]

        # Forward pass
        o_gender, o_hat, o_bag, o_top_color, o_bottom_color = model(X)
        o_gender, o_hat, o_bag, o_top_color, o_bottom_color = output_activations[0](o_gender).squeeze(), output_activations[1](o_hat).squeeze(), output_activations[2](o_bag).squeeze(), output_activations[3](o_top_color).squeeze(), output_activations[4](o_bottom_color).squeeze()

        # Calcola l'accuratezza binaria per i compiti di genere, cappello e borsa
        acc_g = ((o_gender.detach() > 0.5) == y_gender.detach())
        acc_h = ((o_hat.detach() > 0.5) == y_hat.detach())
        acc_b = ((o_bag.detach() > 0.5) == y_bag.detach())

        # Trova la classe predetta
        predicted_classes_tc = torch.argmax(o_top_color, dim=0)
        predicted_classes_bc = torch.argmax(o_bottom_color, dim=0)

        # Calcola l'accuratezza di classe per i compiti di colore superiore e inferiore
        acc_tc = (predicted_classes_tc == y_top_color)
        acc_bc = (predicted_classes_bc == y_bottom_color)

        # Aggiorna il numero totale di classificazioni corrette per ogni compito
        correct_gender += acc_g.item()
        correct_hat += acc_h.item()
        correct_bag += acc_b.item()
        correct_top_color += acc_tc.item()
        correct_bottom_color += acc_bc.item()

# Calcola l'accuratezza per ogni compito
accuracy_gender = correct_gender / total_samples
accuracy_hat = correct_hat / total_samples
accuracy_bag = correct_bag / total_samples
accuracy_top_color = correct_top_color / total_samples
accuracy_bottom_color = correct_bottom_color / total_samples

# Stampa i risultati
print("Accuracy - Gender:", accuracy_gender)
print("Accuracy - Hat:", accuracy_hat)
print("Accuracy - Bag:", accuracy_bag)
print("Accuracy - Top Color:", accuracy_top_color)
print("Accuracy - Bottom Color:", accuracy_bottom_color)


